# Section 9: Quantum Information - Practice Questions

**Exam Weight**: 3% (~2 questions) | **Difficulty**: Medium-High | **Must Master**: ✅✅

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| T gate Clifford | T is Clifford | T is NOT Clifford! (only H, S, CNOT, Paulis) |
| Operator equality | `op1 == op2` | Use `op1.equiv(op2)` for global phase tolerance |
| Fidelity range | [0, ∞) | [0, 1] - 1 = identical states |
| Pauli ordering | `"XYZ"` = X⊗Y⊗Z | RIGHT-TO-LEFT: Z on q0, Y on q1, X on q2 |
| State fidelity | `fidelity(state1, state2)` | `state_fidelity(state1, state2)` |
| Process fidelity | For states | `process_fidelity()` for operators/channels |

> 📖 See section_9_quantum_info/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Quantum Information (`quantum_info_advanced.ipynb`)

#### Operator Class
- **Creation**: `Operator(circuit)`, `Operator(matrix)`
- **Equivalence**: `op1.equiv(op2)` - ignores global phase!
- **Composition**: `op1.compose(op2)`, `op1 @ op2`
- **Tensor product**: `op1.tensor(op2)`, `op1 ^ op2`
- **Properties**: `.dim`, `.is_unitary()`, `.data`

#### Statevector Class
- **Creation**: `Statevector(circuit)`, `Statevector([1, 0])`
- **Operations**: `.evolve(operator)`, `.tensor(other)`
- **Probabilities**: `.probabilities()`, `.probabilities_dict()`
- **Drawing**: `.draw('bloch')`, `.draw('latex')`

#### DensityMatrix Class
- **Creation**: `DensityMatrix(circuit)`, `DensityMatrix(matrix)`
- **Mixed states**: Non-pure quantum states
- **Purity**: `dm.purity()` - 1 for pure, <1 for mixed
- **Trace operations**: `.partial_trace(qubits)`

#### Clifford Circuits
- **Clifford gates**: H, S, CNOT, X, Y, Z (NOT T!)
- **Clifford class**: `Clifford(circuit)`
- **Efficient simulation**: Stabilizer formalism

#### Fidelity Measures
- **State fidelity**: `state_fidelity(state1, state2)` → [0, 1]
- **Process fidelity**: `process_fidelity(channel, target)`
- **Average gate fidelity**: `average_gate_fidelity(channel, target)`

#### Quantum Channels
- **Kraus operators**: `Kraus(ops)`
- **SuperOp**: Superoperator representation
- **Choi matrix**: `Choi(channel)`
- **PTM**: Pauli transfer matrix

#### SparsePauliOp
- **Creation**: `SparsePauliOp('ZZ')`, `SparsePauliOp.from_list([...])`
- **Pauli ordering**: RIGHT-TO-LEFT (rightmost = q0)
- **Operations**: `+`, `@`, `.compose()`

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import Clifford, Statevector, Operator
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.passes import Unroll3qOrMore, Optimize1qGates, CXCancellation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
import numpy as np
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: Dynamic Circuits - if_test()

⚠️ **c_if() is DEPRECATED!** Use `if_test()` instead.

| Old (Deprecated) | New (Preferred) |
|------------------|----------------|
| `gate.c_if(creg, val)` | `with qc.if_test((clbit, val)):` |

### Q1: Basic if_test usage

In [ ]:
# Your solution: Apply X gate only if measurement result is 1

In [ ]:
# Solution Q1
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.measure(0, 0)

# Conditional gate: Apply X to qubit 1 if clbit 0 equals 1
with qc.if_test((qc.clbits[0], 1)):
    qc.x(1)

qc.measure_all()

print("Dynamic circuit with if_test:")
print(qc.draw())

### Q2: if_test with else branch

In [ ]:
# Your solution: Create if-else conditional

In [ ]:
# Solution Q2
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.measure(0, 0)

# if-else pattern
with qc.if_test((qc.clbits[0], 1)) as else_:
    qc.x(1)  # If measured 1
with else_:
    qc.z(1)  # If measured 0

print("Circuit with if-else:")
print(qc.draw())

---
## Part 2: Dynamic Circuits - for_loop()

### Q3: Basic for_loop

In [ ]:
# Your solution: Apply gate multiple times using for_loop

In [ ]:
# Solution Q3
qc = QuantumCircuit(1)

# Apply T gate 4 times (T^4 = Z)
with qc.for_loop(range(4)):
    qc.t(0)

print("Circuit with for_loop (T gate 4x):")
print(qc.draw())

# Verify: T^4 = Z
from qiskit.quantum_info import Operator
op = Operator(qc)
print(f"\nOperator (should be Z up to global phase):")
print(np.round(op.data, 2))

### Q4: for_loop with loop variable

In [ ]:
# Your solution: Use loop index in circuit

In [ ]:
# Solution Q4
qc = QuantumCircuit(3)

# Apply H to each qubit using loop index
with qc.for_loop(range(3)) as i:
    qc.h(i)

print("Circuit with indexed for_loop:")
print(qc.draw())

---
## Part 3: Dynamic Circuits - switch()

### Q5: Basic switch statement

In [ ]:
# Your solution: Apply different gates based on measurement outcome

In [ ]:
# Solution Q5
qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(2, 'c')
qc = QuantumCircuit(qr, cr)

qc.h([0, 1])
qc.measure([0, 1], [0, 1])

# Switch on classical register value
with qc.switch(cr) as case:
    with case(0):  # 00
        qc.x(0)
    with case(1):  # 01
        qc.y(0)
    with case(2):  # 10
        qc.z(0)
    with case(case.DEFAULT):  # 11
        qc.h(0)

print("Circuit with switch:")
print(qc.draw())

---
## Part 4: Custom PassManager

### Q6: Create simple custom PassManager

In [ ]:
# Your solution: Create PassManager with specific passes

In [ ]:
# Solution Q6
from qiskit.transpiler.passes import (
    Unroll3qOrMore,
    Optimize1qGates,
    CXCancellation
)

# Create custom PassManager
pm = PassManager([
    Unroll3qOrMore(),    # Break down 3+ qubit gates
    Optimize1qGates(),   # Merge single-qubit gates
    CXCancellation(),    # Cancel adjacent CX pairs
])

# Test circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 1)  # Should cancel with above
qc.rz(0.1, 0)
qc.rz(0.2, 0)  # Should merge

print("Original:")
print(qc.draw())
print(f"Depth: {qc.depth()}")

# Apply PassManager
optimized = pm.run(qc)
print("\nOptimized:")
print(optimized.draw())
print(f"Depth: {optimized.depth()}")

### Q7: PassManager with stages

In [ ]:
# Your solution: Build PassManager with multiple stages

In [ ]:
# Solution Q7
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import CouplingMap

# Generate preset pass manager
pm = generate_preset_pass_manager(
    optimization_level=2,
    coupling_map=CouplingMap.from_line(3),
    basis_gates=['cx', 'rz', 'sx', 'x']
)

# Test circuit
qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc.cx(0, 2)  # Non-adjacent qubits

print("Original:")
print(qc.draw())

result = pm.run(qc)
print("\nTranspiled:")
print(result.draw())

---
## Part 5: Clifford Gates

### Q8: Create Clifford from circuit

In [ ]:
# Your solution: Convert Clifford circuit to Clifford object

In [ ]:
# Solution Q8
# Clifford gates: H, S, CNOT, X, Y, Z (and combinations)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.s(1)

print("Circuit:")
print(qc.draw())

# Create Clifford from circuit
cliff = Clifford(qc)

print("\nClifford representation:")
print(f"Stabilizer tableau shape: {cliff.tableau.shape}")
print(f"\nDestabilizer:")
for row in cliff.destab:
    print(f"  {row}")
print(f"\nStabilizer:")
for row in cliff.stab:
    print(f"  {row}")

### Q9: Clifford to circuit

In [ ]:
# Your solution: Convert Clifford object back to circuit

In [ ]:
# Solution Q9
# Create a Clifford
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
cliff = Clifford(qc)

# Convert back to circuit
qc_from_cliff = cliff.to_circuit()

print("Original circuit:")
print(qc.draw())
print("\nCircuit from Clifford:")
print(qc_from_cliff.draw())

# Verify equivalence
from qiskit.quantum_info import Operator
print(f"\nEquivalent: {Operator(qc).equiv(Operator(qc_from_cliff))}")

---
## Part 6: JobStatus (Reference)

### Q10: JobStatus values

In [ ]:
# Your solution: List all JobStatus values

In [ ]:
# Solution Q10: JobStatus Reference
# Note: This is primarily for IBM Runtime jobs

print("JobStatus Values (for IBM Runtime):")
print("="*40)
print("INITIALIZING - Job is being initialized")
print("QUEUED - Job is queued for execution")
print("VALIDATING - Job is being validated")
print("RUNNING - Job is currently running")
print("CANCELLED - Job was cancelled")
print("DONE - Job completed successfully")
print("ERROR - Job completed with errors")

# For local simulators
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()

sampler = StatevectorSampler()
job = sampler.run([(qc,)])

# Local jobs complete immediately
print(f"\nLocal job status: Completed immediately")
print(f"Result available: {job.result()[0].data.meas.get_counts()}")

---
## Part 7: Practical Examples

### Q11: Teleportation with dynamic circuits

In [ ]:
# Your solution: Implement quantum teleportation using if_test

In [ ]:
# Solution Q11: Quantum Teleportation
qr = QuantumRegister(3, 'q')  # q0: Alice's state, q1-q2: Bell pair
cr = ClassicalRegister(2, 'c')
qc = QuantumCircuit(qr, cr)

# Prepare state to teleport (e.g., |+⟩)
qc.h(0)
qc.barrier()

# Create Bell pair between q1 and q2
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# Bell measurement
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])
qc.barrier()

# Conditional corrections using if_test
with qc.if_test((cr[1], 1)):
    qc.x(2)
with qc.if_test((cr[0], 1)):
    qc.z(2)

print("Quantum Teleportation Circuit:")
print(qc.draw())

### Q12: Repeat-until-success pattern

In [ ]:
# Your solution: Create pattern that retries until successful measurement

In [ ]:
# Solution Q12: Repeat-until-success (simplified)
qc = QuantumCircuit(1, 1)

# Initialize with some probability of success
qc.ry(np.pi/3, 0)  # Biased towards |0⟩
qc.measure(0, 0)

# If measured 1, try again (reset and retry)
with qc.if_test((qc.clbits[0], 1)):
    qc.reset(0)
    qc.ry(np.pi/3, 0)

qc.measure(0, 0)

print("Repeat-until-success pattern:")
print(qc.draw())

---
## ✅ Section 9 Checklist

**Dynamic Circuits - if_test**:
- [ ] `with qc.if_test((clbit, value)):` - conditional block
- [ ] `with qc.if_test(...) as else_:` + `with else_:` - if-else
- [ ] Replaces deprecated `c_if()`

**Dynamic Circuits - for_loop**:
- [ ] `with qc.for_loop(range(n)):` - repeat n times
- [ ] `with qc.for_loop(range(n)) as i:` - with index

**Dynamic Circuits - switch**:
- [ ] `with qc.switch(creg) as case:` - switch on classical value
- [ ] `with case(value):` - specific case
- [ ] `with case(case.DEFAULT):` - default case

**PassManager**:
- [ ] `PassManager([pass1, pass2, ...])` - custom passes
- [ ] `pm.run(circuit)` - apply passes
- [ ] `generate_preset_pass_manager(opt_level, ...)` - standard presets

**Clifford**:
- [ ] `Clifford(circuit)` - from circuit
- [ ] `cliff.to_circuit()` - to circuit
- [ ] Only works with Clifford gates (H, S, CX, X, Y, Z)

**JobStatus** (IBM Runtime):
- [ ] INITIALIZING, QUEUED, VALIDATING, RUNNING
- [ ] CANCELLED, DONE, ERROR

---
## Part 1: Dynamic Circuits - Control Flow

| Operation | Usage | Pattern |
|-----------|-------|---------|
| `if_test()` | Check measurement result | `with qc.if_test((clbit, value))` |
| `for_loop()` | Fixed iterations | `with qc.for_loop(range(n))` |
| `while_loop()` | Condition-based | `with qc.while_loop(condition)` |
| `switch()` | Multiple branches | `with qc.switch(value)` |

⚠️ **EXAM CRITICAL**: Use `if_test()` NOT deprecated `c_if()`!

### Q1: Basic if_test() - Mid-circuit measurement with conditional gate

In [ ]:
# Your solution: Create circuit with H, measure, apply X conditionally if result is 1

In [ ]:
# Solution Q1
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.measure(0, 0)

# Modern syntax - if_test context manager
with qc.if_test((qc.clbits[0], 1)):  # If classical bit 0 == 1
    qc.x(1)

print("Circuit with if_test:")
print(qc.draw())

### Q2: If-Else Structure

In [ ]:
# Your solution: Create if-else that applies X if 1, Z if 0

In [ ]:
# Solution Q2
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.measure(0, 0)

# If-Else structure using 'as else_:'
with qc.if_test((qc.clbits[0], 1)) as else_:
    qc.x(1)          # If c[0] == 1
with else_:
    qc.z(1)          # Else (c[0] == 0)

print("Circuit with if-else:")
print(qc.draw())

### Q3: for_loop - Runtime iteration

In [ ]:
# Your solution: Create circuit that applies H gate 3 times using for_loop

In [ ]:
# Solution Q3
qc = QuantumCircuit(1)

# for_loop is RUNTIME iteration (on quantum hardware)
# Different from Python for loop (compile-time)
with qc.for_loop(range(3)):
    qc.h(0)

print("Circuit with for_loop (3 iterations):")
print(qc.draw())

### Q4: switch Statement - Multi-way branching

In [ ]:
# Your solution: Create circuit with switch on 2-bit measurement

In [ ]:
# Solution Q4
qc = QuantumCircuit(3, 2)
qc.h([0, 1])
qc.measure([0, 1], [0, 1])

# Switch statement for multi-way branching
with qc.switch(qc.cregs[0]) as case:
    with case(0):   # 00
        qc.x(2)
    with case(1):   # 01
        qc.y(2)
    with case(2):   # 10
        qc.z(2)
    with case(case.DEFAULT):  # 11 (or any other)
        qc.h(2)

print("Circuit with switch:")
print(qc.draw())

### Q5: Quantum Teleportation with Dynamic Corrections

⚠️ **EXAM CLASSIC**: Teleportation uses if_test for corrections!

In [ ]:
# Your solution: Implement teleportation with dynamic corrections

In [ ]:
# Solution Q5 - Quantum Teleportation
def teleportation_dynamic():
    qc = QuantumCircuit(3, 2)
    
    # Prepare state to teleport (example: |+⟩)
    qc.h(0)
    
    # Create Bell pair between q1 and q2
    qc.h(1)
    qc.cx(1, 2)
    
    qc.barrier()
    
    # Bell measurement
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])
    
    qc.barrier()
    
    # Dynamic corrections
    with qc.if_test((qc.clbits[1], 1)):  # If c1 == 1
        qc.x(2)
    with qc.if_test((qc.clbits[0], 1)):  # If c0 == 1
        qc.z(2)
    
    return qc

qc = teleportation_dynamic()
print("Quantum Teleportation with Dynamic Corrections:")
print(qc.draw())

---
## Part 2: Advanced Transpilation

### 6-Stage Transpiler Pipeline
```
INIT → LAYOUT → ROUTING → TRANSLATION → OPTIMIZATION → SCHEDULING
```

| Level | Description | Use Case |
|-------|-------------|----------|
| **0** | No optimization | Debugging |
| **1** | Light | Quick compilation |
| **2** | Medium | Production use |
| **3** | Heavy | Final deployment |

### Q6: generate_preset_pass_manager Full API

In [ ]:
# Your solution: Create pass manager with all options

In [ ]:
# Solution Q6
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()

# Full API (EXAM TESTED!)
pm = generate_preset_pass_manager(
    optimization_level=2,           # 0-3
    backend=backend,                # Target backend
    layout_method='sabre',          # 'trivial', 'dense', 'sabre', 'vf2'
    routing_method='sabre',         # 'basic', 'stochastic', 'sabre'
    scheduling_method='alap',       # 'asap', 'alap'
    seed_transpiler=42              # For reproducibility
)

# Test circuit
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

transpiled = pm.run(qc)
print(f"Original depth: {qc.depth()}")
print(f"Transpiled depth: {transpiled.depth()}")
print("\nTranspiled circuit:")
print(transpiled.draw())

### Q7: Custom PassManager

In [ ]:
# Your solution: Create custom PassManager with optimization passes

In [ ]:
# Solution Q7
from qiskit.transpiler.passes import RemoveBarriers

# Build custom optimization pipeline
custom_pm = PassManager([
    Optimize1qGates(),           # Merge single-qubit gates
    InverseCancellation([]),     # Cancel U·U†
    CommutativeCancellation(),   # Cancel commuting pairs
])

# Test with a circuit that has redundant gates
qc = QuantumCircuit(2)
qc.h(0)
qc.h(0)  # H·H = I, should cancel
qc.x(1)
qc.x(1)  # X·X = I, should cancel
qc.cx(0, 1)

print("Before optimization:")
print(qc.draw())

optimized = custom_pm.run(qc)
print("\nAfter custom PassManager:")
print(optimized.draw())

### Key Transpiler Passes Reference

| Pass | Purpose | Use Case |
|------|---------|----------|
| `Optimize1qGates` | Merge consecutive 1q gates | Reduce gate count |
| `InverseCancellation` | Cancel gate-inverse pairs | X·X = I |
| `CommutativeCancellation` | Cancel commuting gates | ZZ where possible |
| `UnitarySynthesis` | Decompose arbitrary unitaries | Custom gates |
| `RemoveBarriers` | Remove barrier instructions | Optimization |

⚠️ **DEPRECATED**: `CXCancellation` - Use `InverseCancellation` instead!

**Pass Order Matters**: Optimize1qGates → InverseCancellation → Optimize1qGates

### Q8: Compare Optimization Levels

In [ ]:
# Your solution: Transpile same circuit at levels 0, 1, 2, 3 and compare

In [ ]:
# Solution Q8
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()

qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc.cx(0, 2)  # Long-range connection (needs SWAPs)
qc.cx(1, 2)
qc.measure_all()

print(f"Original: depth={qc.depth()}, size={qc.size()}")

for level in range(4):
    transpiled = transpile(qc, backend, optimization_level=level, seed_transpiler=42)
    print(f"Level {level}: depth={transpiled.depth()}, size={transpiled.size()}")

---
## Part 3: Primitives V2 Runtime

### PUB Format (CRITICAL!)

⚠️ **EXAM TRAP**: PUB must be TUPLE `(circuit,)` NOT LIST `[circuit]`

```python
# Sampler PUB
sampler_pub = (circuit,)           # Single circuit - TUPLE!
batch_pubs = [(qc1,), (qc2,)]      # Multiple circuits

# Estimator PUB
estimator_pub = (circuit, observable)
param_pub = (circuit, observable, parameter_values)
```

### Q9: Correct PUB Format for Sampler

In [ ]:
# Your solution: Show correct vs incorrect PUB format

In [ ]:
# Solution Q9
from qiskit.primitives import StatevectorSampler

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = StatevectorSampler()

# ✅ CORRECT - Tuple format
correct_pub = (qc,)
result = sampler.run([correct_pub]).result()
counts = result[0].data.meas.get_counts()
print("✅ Correct PUB format (tuple):")
print(f"Counts: {counts}")

# ❌ WRONG - Would fail with actual runtime
# wrong_pub = [qc]  # This is a list, NOT a tuple!
print("\n❌ NEVER use [circuit] - must be (circuit,)")

### Q10: Correct PUB Format for Estimator

In [ ]:
# Your solution: Create Estimator PUB with observable

In [ ]:
# Solution Q10
from qiskit.quantum_info import SparsePauliOp

# Bell state circuit (no measurement for Estimator!)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Observable
observable = SparsePauliOp.from_list([("ZZ", 1.0)])

estimator = StatevectorEstimator()

# Estimator PUB format: (circuit, observable)
pub = (qc, observable)
result = estimator.run([pub]).result()

print("Estimator PUB format: (circuit, observable)")
print(f"⟨ZZ⟩ = {result[0].data.evs}")  # Should be 1.0 for Bell state

### Session vs Batch (Conceptual - no hardware needed)

| Mode | Use Case | Example |
|------|----------|---------|
| **Session** | Sequential/iterative | VQE, QAOA |
| **Batch** | Parallel/independent | Parameter sweeps |

```python
# Session: Jobs depend on each other
with Session(backend=backend) as session:
    sampler = SamplerV2(mode=session)  # Note: mode= not session=!
    job1 = sampler.run([qc1])
    result1 = job1.result()  # Wait for result
    job2 = sampler.run([qc2])  # Use result1 to inform qc2

# Batch: Jobs are independent
with Batch(backend=backend) as batch:
    sampler = SamplerV2(mode=batch)
    job1 = sampler.run([qc1])  # Runs in parallel
    job2 = sampler.run([qc2])  # Runs in parallel
```

---
## Part 4: Job Management (EXAM CRITICAL!)

### JobStatus Enum

```python
from qiskit.providers import JobStatus

JobStatus.INITIALIZING  # Job is being initialized
JobStatus.QUEUED        # Job is waiting in queue
JobStatus.VALIDATING    # Job is being validated
JobStatus.RUNNING       # Job is currently executing
JobStatus.DONE          # Job completed successfully ✓
JobStatus.ERROR         # Job encountered an error
JobStatus.CANCELLED     # Job was cancelled
```

⚠️ **EXAM TRAP**: It's `JobStatus.DONE` NOT `JobStatus.FINISHED`!

### Q11: JobStatus Enum Values

In [ ]:
# Your solution: List all JobStatus values

In [ ]:
# Solution Q11
from qiskit.providers import JobStatus

print("JobStatus Enum Values:")
for status in JobStatus:
    print(f"  JobStatus.{status.name}")

print("\n⚠️ EXAM TRAP: Use JobStatus.DONE, not FINISHED!")
print(f"\nTerminal states (job.in_final_state() returns True):")
print(f"  - JobStatus.DONE")
print(f"  - JobStatus.ERROR")
print(f"  - JobStatus.CANCELLED")

### Q12: Job Retrieval Pattern

In [ ]:
# Your solution: Show job ID save and retrieval pattern (conceptual)

In [ ]:
# Solution Q12 - Job Retrieval Pattern (Conceptual)
print("""
Job Retrieval Pattern (EXAM TESTED!):

# Step 1: Submit job and save ID
job = sampler.run([qc])
job_id = job.job_id()  # e.g., 'ch2p94bdwm7g008wjz50'
print(f"Save this ID: {job_id}")

# Step 2: Later... retrieve the job
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()
retrieved_job = service.job(job_id)  # ✅ CORRECT

# Step 3: Check status and get results
if retrieved_job.status() == JobStatus.DONE:
    result = retrieved_job.result()
    counts = result[0].data.meas.get_counts()

⚠️ EXAM TRAP:
  ✅ service.job(job_id)     - CORRECT
  ❌ job.retrieve(job_id)    - WRONG
  ❌ service.get_job(job_id) - WRONG
""")

### BasePrimitiveJob Methods

| Method | Return Type | Description |
|--------|-------------|-------------|
| `job_id()` | `str` | Unique job identifier |
| `status()` | `JobStatus` | Current job status |
| `result()` | `PrimitiveResult` | Get results (blocks until complete) |
| `cancel()` | `None` | Cancel a running job |
| `done()` | `bool` | True if job finished |
| `running()` | `bool` | True if currently running |
| `in_final_state()` | `bool` | True if DONE, ERROR, or CANCELLED |

---
## Part 5: Quantum Information

### Clifford Circuits

**Clifford Set**: {H, S, CNOT} - efficiently simulatable (Gottesman-Knill theorem)

**Clifford Hierarchy**:
- Level 1: Pauli gates {I, X, Y, Z}
- Level 2: Clifford gates {H, S, CNOT} - map Paulis to Paulis
- Level 3: T gate, Toffoli - needed for universality

### Q13: Clifford Circuit Simulation

In [ ]:
# Your solution: Create Clifford from circuit and compose

In [ ]:
# Solution Q13
from qiskit.quantum_info import Clifford

# Create a Clifford circuit (only H, S, CNOT allowed)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.s(1)

# Convert to Clifford object
cliff = Clifford.from_circuit(qc)
print("Clifford from circuit:")
print(cliff)

# Random Clifford
random_cliff = Clifford.random(num_qubits=2)
print("\nRandom 2-qubit Clifford:")
print(random_cliff)

# Compose Cliffords
composed = cliff.compose(random_cliff)
print("\nComposed Clifford:")
print(composed)

### Q14: Operator Class - Circuit Equivalence

In [ ]:
# Your solution: Use Operator to check if two circuits are equivalent

In [ ]:
# Solution Q14
from qiskit.quantum_info import Operator

# Two circuits that should be equivalent
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.h(0)  # H·H = I

qc2 = QuantumCircuit(1)
# Empty circuit (identity)

# Convert to Operators
op1 = Operator(qc1)
op2 = Operator(qc2)

print("Circuit 1 (H·H):")
print(qc1.draw())
print("\nCircuit 2 (empty):")
print(qc2.draw())

# Check equivalence
print(f"\nExact equality (op1 == op2): {op1 == op2}")
print(f"Equivalent up to global phase (op1.equiv(op2)): {op1.equiv(op2)}")

### Q15: State Fidelity

In [ ]:
# Your solution: Calculate fidelity between two quantum states

In [ ]:
# Solution Q15
from qiskit.quantum_info import Statevector, state_fidelity

# Create two states
qc1 = QuantumCircuit(1)
qc1.h(0)  # |+⟩ state

qc2 = QuantumCircuit(1)
qc2.h(0)
qc2.z(0)  # |-⟩ state

qc3 = QuantumCircuit(1)
qc3.h(0)  # |+⟩ state (same as qc1)

state1 = Statevector.from_instruction(qc1)
state2 = Statevector.from_instruction(qc2)
state3 = Statevector.from_instruction(qc3)

print(f"|+⟩ state: {state1}")
print(f"|-⟩ state: {state2}")

# Fidelity: F = |⟨ψ|φ⟩|²
print(f"\nFidelity(|+⟩, |-⟩) = {state_fidelity(state1, state2)}")  # 0 - orthogonal
print(f"Fidelity(|+⟩, |+⟩) = {state_fidelity(state1, state3)}")  # 1 - identical

---
## 🎓 Exam Practice Questions

### EQ1: What is the correct PUB format for Sampler?

**A)** `[circuit]`  
**B)** `(circuit,)`  
**C)** `{circuit}`  
**D)** `circuit`  

<details>
<summary>Click for answer</summary>

**Answer: B** - PUB must be a tuple: `(circuit,)`. Lists are NOT valid PUB format.
</details>

### EQ2: What is the correct import for runtime primitives?

**A)** `from qiskit.primitives import SamplerV2`  
**B)** `from qiskit_ibm_runtime import SamplerV2`  
**C)** `from qiskit.runtime import SamplerV2`  
**D)** `from qiskit import SamplerV2`  

<details>
<summary>Click for answer</summary>

**Answer: B** - Runtime primitives are imported from `qiskit_ibm_runtime`.
</details>

### EQ3: When should you use Session mode?

**A)** For parallel independent circuits  
**B)** For iterative algorithms like VQE  
**C)** For single circuit execution  
**D)** Never use Session mode  

<details>
<summary>Click for answer</summary>

**Answer: B** - Session mode is for iterative algorithms where results inform subsequent jobs (VQE, QAOA).
</details>

### EQ4: Which JobStatus indicates a job completed successfully?

**A)** `JobStatus.RUNNING`  
**B)** `JobStatus.QUEUED`  
**C)** `JobStatus.DONE`  
**D)** `JobStatus.FINISHED`  

<details>
<summary>Click for answer</summary>

**Answer: C** - `JobStatus.DONE` indicates successful completion. `FINISHED` is not a valid JobStatus value.
</details>

### EQ5: How do you retrieve a previously submitted job?

**A)** `job.retrieve(job_id)`  
**B)** `service.get_job(job_id)`  
**C)** `service.job(job_id)`  
**D)** `QiskitRuntimeService.job(job_id)`  

<details>
<summary>Click for answer</summary>

**Answer: C** - Use `service.job(job_id)` where `service` is a QiskitRuntimeService instance.
</details>

### EQ6: Which method returns True if a job is in DONE, ERROR, or CANCELLED state?

**A)** `job.done()`  
**B)** `job.completed()`  
**C)** `job.in_final_state()`  
**D)** `job.finished()`  

<details>
<summary>Click for answer</summary>

**Answer: C** - `job.in_final_state()` returns True if the job is in any terminal state (DONE, ERROR, CANCELLED).
</details>

### EQ7: Which pass should you use to cancel gate-inverse pairs (like X·X)?

**A)** `CXCancellation`  
**B)** `InverseCancellation`  
**C)** `Optimize1qGates`  
**D)** `CommutativeCancellation`  

<details>
<summary>Click for answer</summary>

**Answer: B** - `InverseCancellation` cancels gate-inverse pairs. `CXCancellation` is DEPRECATED!
</details>

### EQ8: What is the Clifford gate set?

**A)** {X, Y, Z}  
**B)** {H, S, CNOT}  
**C)** {H, T, CNOT}  
**D)** {RX, RY, RZ}  

<details>
<summary>Click for answer</summary>

**Answer: B** - The Clifford gates are {H, S, CNOT}. These can be efficiently simulated classically (Gottesman-Knill theorem).
</details>

### EQ9: What is the modern syntax for conditional operations?

**A)** `qc.x(0).c_if(c, 1)`  
**B)** `with qc.if_test((c[0], 1)): qc.x(0)`  
**C)** `qc.conditional(c, 1).x(0)`  
**D)** `qc.if(c == 1).x(0)`  

<details>
<summary>Click for answer</summary>

**Answer: B** - Use `with qc.if_test((clbit, value)):` context manager. The `c_if()` method is DEPRECATED.
</details>

### EQ10: What does `state_fidelity()` return for orthogonal states?

**A)** 1.0  
**B)** 0.5  
**C)** 0.0  
**D)** -1.0  

<details>
<summary>Click for answer</summary>

**Answer: C** - State fidelity is 0 for orthogonal states and 1 for identical states. F = |⟨ψ|φ⟩|² ∈ [0, 1].
</details>

---
## 📋 Critical Exam Traps Summary

| Trap | Wrong | Correct |
|------|-------|---------|
| PUB format | `[circuit]` | `[(circuit,)]` |
| Job status | `JobStatus.FINISHED` | `JobStatus.DONE` |
| Job retrieval | `job.retrieve(id)` | `service.job(job_id)` |
| Primitives import | `qiskit.primitives` | `qiskit_ibm_runtime` |
| Conditional | `qc.x(0).c_if(c,1)` | `with qc.if_test((c[0],1))` |
| Session mode | `session=session` | `mode=session` |
| Gate cancellation | `CXCancellation` | `InverseCancellation` |

---
## ✅ Notebook Complete

You've practiced:
1. ✅ Dynamic Circuits (if_test, for_loop, while_loop, switch)
2. ✅ Advanced Transpilation (PassManager, generate_preset_pass_manager)
3. ✅ Primitives V2 (PUB format, Session vs Batch)
4. ✅ Job Management (JobStatus, job retrieval)
5. ✅ Quantum Information (Clifford, Operator, Fidelity)

**Next Steps:**
- Review section_9 README for theoretical foundations
- Practice the exam questions until you get 100%
- Run the code cells to build muscle memory

*Last Updated: December 19, 2025*